In [1]:
from rtt_tools import dump_data
from rtt_tools.dump_data import *  # pussy died because of this wildcard import
from rtt_tools.export import *

logger = logging.getLogger(__name__)
coloredlogs.CHROOT_FILES = []
coloredlogs.install(level=logging.INFO, use_chroot=False)

In [ ]:
from rtt_tools.cleaner import Cleaner
cl = Cleaner()
cl.exp_id_low = 284942
cl.main()

In [2]:
loader = dump_data.Loader()
loader.thr_stest_load = 3000
exporter = Exporter(loader)

In [3]:
exp_id_list = list(range(260259, 260259+2000))

In [ ]:
exporter.load({'no_pvals':False, 'only_pval_cnt': True, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/tmp', fname='rtt-results-syso-diehgarder-t05-t06-full.json')
print(exporter.fname)


In [ ]:
ex_start = 261759 + 100  # original start: 260259
ex_end = 260259 + 2000
ex_step = 100

for exr in range(ex_start, ex_end, ex_step):
    loader = dump_data.Loader()
    loader.thr_stest_load = 1000
    exporter = Exporter(loader)
    exp_id_list = list(range(exr, exr + ex_step))
    
    exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
    exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t05-t06-%s.json' % (exr,))
    print(exporter.fname)
    

In [ ]:
exr = 262265
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 266975+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t07-nist-qrng-%s.json' % (exr,))
print(exporter.fname)

In [ ]:
exr = 271496
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 283464+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t08-aes-nist-%s.json' % (exr,))
print(exporter.fname)

In [ ]:
exr = 283466
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 284515+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t10-aes-nist-stdconf-%s.json' % (exr,))
print(exporter.fname)

In [2]:
from rtt_tools.gen.max_rounds import FUNC_DB, FuncDb, FuncInfo

In [3]:
import json
js = json.load(open('/tmp/rtt-results-full7.json'))

In [4]:
print(json.dumps(js[2829322], indent=2))
#for rix, rec in enumerate(js):
#    if 'lowmc' in rec['data_type']:
#        print(rec['data_type'])
#     exp_info = loader.break_exp_ph4(rec['data_type'])
#     if not exp_info:
#         print('Could not parse %s' % rec['data_type'])
#         continue

{
  "test": "TestU01 Rabbit|swalk_RandomWalk1",
  "subtest": "1024|1024|1|97656|0|32",
  "subtest_type": "L0|L1|N|n|r|s",
  "variant": "100000000|1",
  "variant_type": "Bit NB|Repetitions",
  "variant_id": 36725269,
  "exid": 313155,
  "data_type": "PH4-SM-03-Lesamnta-t:hash-r:3-b:32-s:100MiB-e:2-i:ctr-256sbit-offset-2",
  "aux_res": null,
  "subs": [
    {
      "sid": 80976007,
      "idx": 1,
      "pvals": [],
      "stats": [
        {
          "name": "Chi-square(C)",
          "value": 0.5099999904632568,
          "pass": true
        },
        {
          "name": "Chi-square(H)",
          "value": 0.006800000190734863,
          "pass": true
        },
        {
          "name": "Chi-square(J)",
          "value": 0.5099999904632568,
          "pass": true
        },
        {
          "name": "Chi-square(M)",
          "value": 0.4099999964237213,
          "pass": true
        },
        {
          "name": "Chi-square(R)",
          "value": 0.1899999976158142,
       

In [5]:
loader = dump_data.Loader()

def exp_info_to_rec(exp_info):
    return collections.OrderedDict([
        ('tp', exp_info.fnc_type),
        ('f', exp_info.fnc_name),
        ('s', exp_info.size),
        ('m', exp_info.meth),
        ('r', exp_info.fnc_round),
        ('e', exp_info.id),
    ])

def name_to_dict(name):  # js[0]['data_type']
    exp_info = loader.break_exp_ph4(name)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(name)
    if exp_info is None or exp_info.fnc is None:
        return None
    return exp_info_to_rec(exp_info)


In [6]:
funtype = {}  # fun -> type
funrounds = collections.defaultdict(lambda: set())  # fun -> set of rounds tested
methsize = {}
testvariant = {}
datatype = {}
fnames = set()
test_db = collections.defaultdict(lambda: 0)
test_db_inv = collections.defaultdict(lambda: 0)
test_db_ctr = set()

# func_name -> (meth, size) -> round -> test:var:subtest -> [pass, fail, fail]  # 3 seed results
funcres = collections.defaultdict(                   # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round
            lambda: collections.defaultdict(         # test-variant-subtest
                lambda: collections.OrderedDict([        
                    ('info', None),
                    ('recidx', None),
                    ('exid', None),
                    ('exs', [None]*3),
                    ('exs_info', [None]*3),
])))))


for rix, rec in enumerate(js):
    data_type = rec['data_type']
    data_type = data_type.replace('stream:cipher', 'stream_cipher')
    exp_info = loader.break_exp_ph4(data_type)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(data_type)
    if not exp_info or not exp_info.fnc:
        print('Could not parse %s' % data_type)
        continue
    #if rix > 1000:
    #    break
        
    funtype[exp_info.fnc_name] = exp_info.fnc_type
    funrounds[exp_info.fnc_name].add(exp_info.fnc_round)

    meth_size = '%s:%s' % (exp_info.meth, exp_info.size)
    if 'testmpc' in data_type:
        meth_size = '%s:%s:%s' % (exp_info.meth, exp_info.spread, exp_info.size)
    
    methsize[meth_size] = [exp_info.meth, exp_info.size, exp_info.spread]
    cround = exp_info.fnc_round  # TODO: not always int
    datatype[data_type] = exp_info_to_rec(exp_info)
    
    test_desc = f'{rec["test"]}##[{rec["variant_type"]}][{rec["variant"]}]##[{rec["subtest_type"]}][{rec["subtest"]}]'
    testvariant[test_desc] = collections.OrderedDict([        
        ('variant_type', rec["variant_type"]),
        ('variant', rec["variant"]),
        ('variant_id', rec["variant_id"]),
        ('subtest_type', rec["subtest_type"]),
        ('subtest', rec["subtest"]),
        ('exid', rec["exid"]),
    ])
    
    test_db_ctr.add(test_desc)
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['exid'] = rec['exid']
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['recidx'] = rix
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['info'] = data_type
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['einfo'] = exp_info
    
    # iterate over subtests
    nsubs_passed = 0
    for sub in rec['subs']:
        is_passed = sub['stats'][0]['pass']
        nsubs_passed += int(is_passed)
        
    nsubs = len(rec['subs'])
    npass_ratio = nsubs_passed / nsubs
    variant_passed = npass_ratio >= 0.99  # tune this ratio! Ratio of passed subtests to consider test passed
    
    if exp_info.id >= 3:
        print(f'!! Out of range {exp_info.id} for {data_type}')
        continue
        
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['exs'][exp_info.id] = variant_passed
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['exs_info'][exp_info.id] = collections.OrderedDict([
        ('exid', rec['exid']),
        ('recidx', rix),
        ('info', data_type),
        ('einfo', exp_info),
        ('nsubs', nsubs),
        ('nsubsp', nsubs_passed),
        ('nsubsr', npass_ratio),
    ])
    # print('#: %s, pass: %s, pct: %2.7f' % (nsubs, nsubs_passed, 100*nsubs_passed/nsubs))
    
# Reindex tests so we have stable IDs
all_tests = sorted(list(test_db_ctr))
for ix, test in enumerate(all_tests):
    test_db[test] = ix
    test_db_inv[ix] = test
    
#print(funrounds, funtype, funcres)
#print(json.dumps(funcres, indent=2))
print('Proc1 done')

Proc1 done


In [9]:
# funcres["AES"]["ctr:10485760"][3]
# Analyze function -> meth-size -> round; all tests, reject if 2/3 rejected

# All experiments computed
data_type_set = set()

# func_name -> (meth, size) -> round -> {summary dict}
funcres_agg = collections.defaultdict(               # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round        
            lambda: collections.OrderedDict([        
                ('num_tests', 0),
                ('num_rej', 0),
                ('rejects', False),
                ('lib_num_tests', 0),
                ('lib_num_rej', 0),
                ('lib_rejects', False),
                ('info', None),
                ('einfo', None),
                ('exid', None),
                ('exs_info', collections.defaultdict(lambda: collections.OrderedDict([
                    ('exids', set()),
                    ('num_tests', 0),
                    ('num_rej', 0),
                    ('lib_rejects', 0),
                    ('rejs', set()),
                    ('all_tests', set()),
                ]))),
])))) 

# Liberal thresholding - both of the above
liberal_threshold_ratio = 0.011  # at least 0.011 rejection rate
liberal_threshold_const = 3  # at least 3 tests
conservative_ratio = 0.01  # num_rej / num_tests; num_rej += 1 if the same tests was rejected at least twice; # TUNE THIS!

ignore_random_excursions = True
ignore_testmpc = False

uncomps = 0
uncomps_funcs = collections.Counter()
uncomps_set = set()
uncomps_bat = set()
rerun_cmds = collections.defaultdict(lambda: set())

bat_map = {
    'NIST Statistical Testing Suite': '--nist_sts',
    'Dieharder': '--dieharder',
    'TestU01 Alphabit': '--tu01_alphabit',
    'TestU01 Small Crush': '--tu01_crush',
    'TestU01 Rabbit': '--tu01_rabbit',
    'TestU01 Block Alphabit': '--tu01_blockalphabit',
    'booltest_1': '--booltest-1',
    'booltest_2': '--booltest-2',
}

expfiles = []
if os.path.exists('expfiles.json'):
    with open('expfiles.json') as fh:
        expfiles = json.load(fh)
        
        
def find_expfile(tpl, expfiles):
    for fname in expfiles:
        if tpl in fname:
            yield fname

            
for fname in funcres:
    for meth_size in funcres[fname]:
        for cround in funcres[fname][meth_size]:
            ctests = funcres[fname][meth_size][cround]  # all tests map
            crec = funcres_agg[fname][meth_size][cround]  # resulting dict
            
            for ctest in ctests:
                data_type_set.add(ctests[ctest]["info"])
                
                for resix, resex in enumerate(ctests[ctest]['exs']):
                    if resex is None:
                        uncomps += 1
                        uncomps_funcs[fname] += 1
                        ename = ctests[ctest]["info"]
                        
                        if ignore_random_excursions and 'Random Excursions' in ctest:
                            continue
                        if ignore_testmpc and 'testmpc' in ename:
                            continue
                            
                        if '-e:' in ename:
                            ename2 = re.sub(r'-e:[\d]-', f'-e:{resix}-', ename)
                            ename2 = re.sub(r'-i:([\w]+?)-.*', '-i:\\1', ename2)
                            ename2 = re.sub(r'^PH4-SM-[\d]+-', '', ename2)
                            fnames = list(find_expfile('-' + ename2.replace(':', '_'), expfiles))
                            if len(fnames) != 1:
                                print(f'! bad fname count: {fnames}, {fname} {meth_size} {cround} {ctest} | {ename} | resix: {resix}')
                                continue
                            ename = fnames[0]
                            ename = ename.replace('.json', '')
                            ename = ename.replace('_', ':')

                        elif 'testmpc' in ename:
                            ename = re.sub(r'-inp-([\w]+?)([\d]+)-', '-inp-\\1!XXXX', ename)
                            ename = ename.replace('!XXXX', '%02d-' % resix)
                            ename = ename.replace('.json', '')
                            ename = re.sub(r'^.*-testmpc', 'testmpc', ename)
                        
                        uncomps_set.add(ename)
                        uncomps_bat.add(ctest.split('|')[0])
                        rerun_cmds[ename].add(ctest.split('|')[0])
                        print(f'Uncomputed: {fname}-{meth_size}-{cround}-{ctest}-{ctests[ctest]["exs"]}-{ctests[ctest]["recidx"]}, {ename}')
                    
                if None in ctests[ctest]['exs']:                    
                    # print(json.dumps(js[ctests[ctest]["recidx"]], indent=2))
                    continue
                    
                # Liberal interpretation: threshold of tests is enough to tell that we reject the generator.
                crec['lib_num_tests'] += 1
                
                # Iterating over seed variant runs for given fname-methsize-round-ctest
                exs_info = ctests[ctest]['exs_info']
                for ix, exsinfo in enumerate(exs_info):
                    if exsinfo is None:
                        continue  # test was not found for this exp.id (seed alternation)
                        
                    # Single test rejects single seed stream
                    is_rejected = int(not ctests[ctest]['exs'][ix]) if ctests[ctest]['exs'][ix] is not None else 0 
                    ecrec = crec['exs_info'][ix]  # ecrec ~ funcres_agg[fname][meth_size][cround]['exs_info'][ix]
                    ecrec['exids'].add(ctests[ctest]['exs_info'][ix]['exid'])
                    ecrec['num_tests'] += 1
                    ecrec['num_rej'] += is_rejected
                    ecrec['all_tests'].add(test_db[ctest])
                    if is_rejected:
                        ecrec['rejs'].add(test_db[ctest])
                        
                    # With each update recompute lib_rejects criteria based on the current test results.    
                    ecrec['lib_rejects'] = ecrec['num_rej'] >= liberal_threshold_const and (ecrec['num_rej'] / ecrec['num_tests']) >= liberal_threshold_ratio
                    
                    # Global liberal view, summarizind seed runs
                    crec['lib_num_tests'] += 1
                    crec['lib_num_rej'] += is_rejected
                    
                # Conservative interpretation: We reject if we can reproduce the *same test* at least 2 times. 
                # 'num_rej' is then number of tests that rejected the same input at least 2 times. 
                is_rejected = sum(ctests[ctest]['exs']) <= 1  # counting passes, 1 or less passes -> rejected (2+ rejections)
                crec['num_tests'] += 1  # +1 for each test computed for fnc-methsize-round
                crec['num_rej'] += int(is_rejected)  # +1 for each rejection by a single test for fnc-methsize-round, reproduced on 3 buckets
                crec['info'] = ctests[ctest]['info']
                crec['einfo'] = ctests[ctest]['einfo']
                crec['exid'] = ctests[ctest]['exid']

            if crec['num_tests'] == 0:
                print(f'! zero tests: {fname}-{meth_size}-{cround}')
                continue
                
            cratio = crec['num_rej'] / crec['num_tests']
            crec['rejects'] = cratio >= conservative_ratio
            crec['lib_rejects'] = sum([int(crec['exs_info'][kk]['lib_rejects']) for kk in crec['exs_info'].keys()]) > 1 # 2 or more rejects
            
            
print('\n\n\n\n\n\n\n--------------------\nUncomputed total:', uncomps, 'functions:', uncomps_funcs, ' batteries:', uncomps_bat)
for ename in sorted(list(rerun_cmds.keys())):
    fname = ename.replace(':', '_')
    fname += '.json'
    bats = sorted(list(rerun_cmds[ename]))
    ename2 = ename.replace('-SM-01-', '-SM-04-')
    ename2 = ename2.replace('-SM-02-', '-SM-04-')
    fsize = ('100' if '100MiB' in fname else '10') if 'MiB' in fname else ('100' if '100MB' in fname else '10')
    print(f"submit_experiment {' '.join([bat_map[x] for x in bats])} --name '{ename2}' --cfg '/home/debian/rtt-home/RTTWebInterface/media/predefined_configurations/{fsize}MB.json' --cryptostreams-config '{fname}'")
    
print('-----------------\n\n\nProc2 done') 
print('Num of computed experiments: %s' % (len(data_type_set)))

Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Small Crush|smarsa_BirthdaySpacings##[Repetitions][1]##[N|d|n|p|r|t][1|1073741824|5000000|1|0|2]-[False, None, None]-67282, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Small Crush|smarsa_BirthdaySpacings##[Repetitions][1]##[N|d|n|p|r|t][1|1073741824|5000000|1|0|2]-[False, None, None]-67282, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Small Crush|sknuth_Collision##[Repetitions][1]##[N|Sparse|d|n|r|t][1|TRUE|65536|5000000|0|2]-[False, None, None]-67283, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Small Crush|sknuth_Collision##[Repetitions][1]##[N|Sparse|d|n|r|t][1|TRUE|65536|5000000|0|2]-[False, None, None]-67283, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Small Crush|sknuth_CouponCollec

Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 256-3-1##[Cfg][256-3-1]##[][]-[False, False, None]-2939615, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 384-1-2##[Cfg][384-1-2]##[][]-[False, False, None]-2939634, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 384-3-1##[Cfg][384-3-1]##[][]-[False, False, None]-2939629, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 128-1-3##[Cfg][128-1-3]##[][]-[False, False, None]-2939611, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 128-2-1##[Cfg][128-2-1]##[][]-[True, True, None]-2939616, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-booltest_2|halving 256-1-2##[Cfg][256-1-2]##[][]-[F

Uncomputed: lowmc-s128d-lhw::104857600-50-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][100000000|1]##[L|N|Sparse|n|r|s][38|11|TRUE|9090880|0|32]-[True, True, None]-2161599, testmpc02-lowmc-s128d-bin-raw-r50-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-50-TestU01 Rabbit|snpair_ClosePairsBitMatch##[Bit NB|Repetitions][100000000|1]##[N|n|r|t][1|1562500|0|2]-[True, True, None]-2161600, testmpc02-lowmc-s128d-bin-raw-r50-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-50-TestU01 Rabbit|snpair_ClosePairsBitMatch##[Bit NB|Repetitions][100000000|1]##[N|n|r|t][1|781250|0|4]-[True, True, None]-2161601, testmpc02-lowmc-s128d-bin-raw-r50-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-50-TestU01 Rabbit|svaria_AppearanceSpacings##[Bit NB|Repetitions][100000000|1]##[K|L|N|Q|r|s][3125000|15|1|3125000|0|30]-[True, True, None]-2161602, testmpc02-lowmc-s128d-bin-raw-r50-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-

Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 128-2-1##[Cfg][128-2-1]##[alpha][0.00001]-[True, True, None]-2942869, testmpc01-lowmc-s128a-bin-raw-r2-inp-lhw02-b32-w4-spr--s100MB
Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 128-2-2##[Cfg][128-2-2]##[alpha][0.00001]-[True, True, None]-2942888, testmpc01-lowmc-s128a-bin-raw-r2-inp-lhw02-b32-w4-spr--s100MB
Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 512-3-2##[Cfg][512-3-2]##[alpha][0.00001]-[True, True, None]-2942903, testmpc01-lowmc-s128a-bin-raw-r2-inp-lhw02-b32-w4-spr--s100MB
Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 384-3-2##[Cfg][384-3-2]##[alpha][0.00001]-[True, True, None]-2942889, testmpc01-lowmc-s128a-bin-raw-r2-inp-lhw02-b32-w4-spr--s100MB
Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 512-3-3##[Cfg][512-3-3]##[alpha][0.00001]-[True, True, None]-2942904, testmpc01-lowmc-s128a-bin-raw-r2-inp-lhw02-b32-w4-spr--s100MB
Uncomputed: lowmc-s128a-lhw::104857600-2-booltest_1|v1 384-3-1##[

Uncomputed: lowmc-s128c-lhw::104857600-2-TestU01 Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Repetitions][100000000|0|32|1]##[L|N|Sparse|n|r|s][4|1|FALSE|100000000|0|32]-[True, True, None]-2936239, testmpc01-lowmc-s128c-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128c-lhw::104857600-2-TestU01 Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Repetitions][100000000|0|32|1]##[L|N|Sparse|n|r|s][8|1|FALSE|100000000|0|32]-[True, True, None]-2936240, testmpc01-lowmc-s128c-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128c-lhw::104857600-2-TestU01 Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Repetitions][100000000|0|32|1]##[L|N|Sparse|n|r|s][16|1|FALSE|100000000|0|32]-[True, True, None]-2936241, testmpc01-lowmc-s128c-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128c-lhw::104857600-2-TestU01 Alphabit|sstring_HammingIndep##[Bit NB|Bit R|Bit S|Repetitions][100000000|0|32|1]##[L|N|d|n|r|s][16|1|0|3125000|0|32]-[True, Tru

...

Uncomputed: lowmc-s80b-lhw::104857600-1-TestU01 Rabbit|sstring_AutoCor##[Bit NB|Repetitions][100000000|1]##[N|d|n|r|s][1|2|99999938|0|32]-[True, True, None]-2934577, testmpc01-lowmc-s80b-bin-raw-r1-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s80b-lhw::104857600-1-TestU01 Rabbit|sstring_Run##[Bit NB|Repetitions][100000000|1]##[N|n|r|s][1|20000000|0|32]-[False, False, None]-2934578, testmpc01-lowmc-s80b-bin-raw-r1-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s80b-lhw::104857600-1-TestU01 Rabbit|smarsa_MatrixRank##[Bit NB|Repetitions][100000000|1]##[L|N|k|n|r|s][32|1|32|97656|0|32]-[False, False, None]-2934579, testmpc01-lowmc-s80b-bin-raw-r1-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s80b-lhw::104857600-1-TestU01 Rabbit|smarsa_MatrixRank##[Bit NB|Repetitions][100000000|1]##[L|N|k|n|r|s][320|1|320|976|0|32]-[False, False, None]-2934580, testmpc01-lowmc-s80b-bin-raw-r1-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s80b-lhw::104857600-1-TestU01 Rabbit|smarsa_MatrixRank##[Bit NB|Re

Uncomputed: lowmc-s80b-ctr::104857600-2-Dieharder|RGB Lagged Sum Test##[N-tuple setting|P-sample count][6|1]##[][]-[True, None, None]-138924, testmpc01-lowmc-s80b-bin-raw-r2-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s80b-ctr::104857600-2-Dieharder|RGB Lagged Sum Test##[N-tuple setting|P-sample count][6|1]##[][]-[True, None, None]-138924, testmpc01-lowmc-s80b-bin-raw-r2-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s80b-ctr::104857600-2-Dieharder|RGB Lagged Sum Test##[N-tuple setting|P-sample count][7|1]##[][]-[True, None, None]-138925, testmpc01-lowmc-s80b-bin-raw-r2-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s80b-ctr::104857600-2-Dieharder|RGB Lagged Sum Test##[N-tuple setting|P-sample count][7|1]##[][]-[True, None, None]-138925, testmpc01-lowmc-s80b-bin-raw-r2-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s80b-ctr::104857600-2-Dieharder|RGB Lagged Sum Test##[N-tuple setting|P-sample count][8|1]##[][]-[True, None, None]-138926, testmpc01-lowmc-s80b-bin-raw-r2-inp-ctr01-b16-spr--s100MB


Uncomputed: mimc_hash-S128-lhw:s6ob:104857600-1-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][100000000|1]##[L|N|Sparse|n|r|s][38|11|TRUE|9090880|0|32]-[False, False, None]-235407, testmpc01-S128-pri-raw-r1-inp-lhw02-b31-w4-spr-s6ob-s100MB
Uncomputed: mimc_hash-S128-lhw:s16mb1:104857600-1-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][100000000|1]##[L|N|Sparse|n|r|s][38|11|TRUE|9090880|0|32]-[False, False, None]-218748, testmpc01-S128-pri-raw-r1-inp-lhw02-b31-w4-spr-s16mb1-s100MB
Uncomputed: mimc_hash-S128-lhw:s6mb:104857600-1-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][100000000|1]##[L|N|Sparse|n|r|s][38|11|TRUE|9090880|0|32]-[False, False, None]-225718, testmpc01-S128-pri-raw-r1-inp-lhw02-b31-w4-spr-s6mb-s100MB
Uncomputed: Vision_S45a-lhw:s0mb:10485760-2-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][10000000|1]##[L|N|Sparse|n|r|s][36|2|TRUE|5000000|0|32]-[True, None, True]-2752370, testmpc02-Vision_S45a-bin-

Uncomputed: Vision_S128d-ctr:s18ob:10485760-2-TestU01 Rabbit|smultin_MultinomialBitsOver##[Bit NB|Repetitions][10000000|1]##[L|N|Sparse|n|r|s][36|2|TRUE|5000000|0|32]-[True, True, None]-2763611, testmpc02-Vision_S128d-bin-raw-r2-inp-ctr02-b7-spr-s18ob-s10MB
Uncomputed: Vision_S128d-ctr:s18ob:10485760-2-TestU01 Rabbit|snpair_ClosePairsBitMatch##[Bit NB|Repetitions][10000000|1]##[N|n|r|t][1|156250|0|2]-[True, True, None]-2763612, testmpc02-Vision_S128d-bin-raw-r2-inp-ctr02-b7-spr-s18ob-s10MB
Uncomputed: Vision_S128d-ctr:s18ob:10485760-2-TestU01 Rabbit|snpair_ClosePairsBitMatch##[Bit NB|Repetitions][10000000|1]##[N|n|r|t][1|78125|0|4]-[True, True, None]-2763613, testmpc02-Vision_S128d-bin-raw-r2-inp-ctr02-b7-spr-s18ob-s10MB
Uncomputed: Vision_S128d-ctr:s18ob:10485760-2-TestU01 Rabbit|svaria_AppearanceSpacings##[Bit NB|Repetitions][10000000|1]##[K|L|N|Q|r|s][468750|10|1|468750|0|30]-[True, True, None]-2763614, testmpc02-Vision_S128d-bin-raw-r2-inp-ctr02-b7-spr-s18ob-s10MB
Uncomputed: Visio

! bad fname count: [], BLAKE hw:104857600 4 booltest_1|v1 512-2-2##[Cfg][512-2-2]##[alpha][0.00001] | PH4-SM-05-BLAKE-t:hash-r:4-b:32-s:100MiB-e:0-i:hw-256bit-hw4-offsetidx-0-offset-0-1-2-3-r0.00-vecsize-174792640-100MiB
! bad fname count: [], BLAKE hw:104857600 4 booltest_1|v1 384-2-2##[Cfg][384-2-2]##[alpha][0.00001] | PH4-SM-05-BLAKE-t:hash-r:4-b:32-s:100MiB-e:0-i:hw-256bit-hw4-offsetidx-0-offset-0-1-2-3-r0.00-vecsize-174792640-100MiB
! bad fname count: [], BLAKE hw:104857600 4 booltest_1|v1 512-2-1##[Cfg][512-2-1]##[alpha][0.00001] | PH4-SM-05-BLAKE-t:hash-r:4-b:32-s:100MiB-e:0-i:hw-256bit-hw4-offsetidx-0-offset-0-1-2-3-r0.00-vecsize-174792640-100MiB
! bad fname count: [], BLAKE hw:104857600 4 booltest_1|v1 512-1-3##[Cfg][512-1-3]##[alpha][0.00001] | PH4-SM-05-BLAKE-t:hash-r:4-b:32-s:100MiB-e:0-i:hw-256bit-hw4-offsetidx-0-offset-0-1-2-3-r0.00-vecsize-174792640-100MiB
! bad fname count: [], BLAKE hw:104857600 4 booltest_1|v1 128-1-3##[Cfg][128-1-3]##[alpha][0.00001] | PH4-SM-05-BLAK

In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Liberal thresholding
# TODO: compute higher rounds for liberal interpretation, e.g., Tangle has not enough
use_liberal = True
funcs_no_rounds = set(['MCSSHA3', 'Shabal'])

log_lines = []
table_lines = []
tests_to_print = set()

exids_recomp_margin = 3  # rounds a top of maximal / the latest fail
exids_recompute = collections.defaultdict(  # eid
    lambda: collections.defaultdict(        # meth-size
        lambda: list()))                    # set of rounds to compute


def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [int(text) if text.isdigit() else text.lower()
            for text in _nsre.split(s)]

for fname in sorted(funcres_agg.keys(), key=natural_sort_key):
    fname_low = fname.lower()
    
    # Find info per-fname
    fmeth = list(funcres_agg[fname].keys())
    fround = list(funcres_agg[fname][fmeth[0]].keys()) if fmeth else None
    finfo = funcres_agg[fname][fmeth[0]][fround[0]] if fround else None
    finfo = finfo['einfo'] if finfo and 'einfo' in finfo else None
        
    erec = FUNC_DB.search(fname, FuncInfo.HASH if finfo is not None and finfo.fnc_type == 'hash' else FuncInfo.CIPHER)
    if erec is None and finfo.fnc_type == 'prng':
        erec = FuncInfo(fname, FuncInfo.PRNG, 0, 1, 1)
    
    max_rounds_cur = erec.max_rounds if erec else None
    crypt_rounds_cur = erec.human_broken_rounds if erec else None
    max_rounds_cur_s = ('%3d' % max_rounds_cur) if max_rounds_cur is not None else ' - '
    crypt_rounds_cur_s = ('%3d' % crypt_rounds_cur) if crypt_rounds_cur is not None else ' - '
    max_rnd_broken = None
    max_rnd_broken_meth = collections.defaultdict(lambda: 0)
    max_rnd_meth = collections.defaultdict(lambda: 0)
    
    for meth in sorted(funcres_agg[fname].keys()):
        namemeth = funcres_agg[fname][meth]
        #if '104857600' not in meth: # or 'sac:' in meth:
        #    continue
        
        rounds = list(namemeth.keys())
        for ix, rnd in enumerate(rounds):
            if rnd == '1-0-0':
                rounds[ix] = 1
                namemeth[1] = namemeth['1-0-0']
            if rnd == 'x':
                namemeth[1] = namemeth['x']
                rounds[ix] = 1
            if rnd is None:
                namemeth[1] = namemeth[None]
                rounds[ix] = 1
                
        rounds.sort()
        cexids = [-1]*3
        for rnd in rounds:
            crec = namemeth[rnd]
            cexid = crec['exid']
            max_rnd_meth[meth] = max(max_rnd_meth[meth], rnd)
            
            for ixx in crec['exs_info'].keys():
                cexids[ixx] = max(cexids[ixx], max(crec['exs_info'][ixx]['exids']))
            
            cur_rejects = crec['lib_rejects'] if use_liberal else crec['rejects']
            cnum_rej = crec["lib_num_rej"] if use_liberal else crec["num_rej"]
            cnum_tests = crec["lib_num_tests"] if use_liberal else crec["num_tests"]
            
            con_rejects = crec['rejects']
            con_num_rej = crec["num_rej"]
            con_num_tests = crec["num_tests"]
            
            # Conservative view on rejection - same test fails at least twice on 3 seed streams
            # Liberal: X tests per seed run. Needs to be set properly.
            if cur_rejects and (max_rnd_broken is None or rnd > max_rnd_broken):
                max_rnd_broken = rnd
            if cur_rejects and (max_rnd_broken_meth[meth] is None or rnd > max_rnd_broken_meth[meth]):
                max_rnd_broken_meth[meth] = rnd
                
            # Detailed view on which tests failed in which experiments
            # In conservative interpretation, we require the same test to fail at least twice / 3 runs.
            # 'num_rej' thus does not have to match rejected tests in particular seed run 
            # (e.g., if test IDs are disjunct, 'num_rej' is 0)
            rejs_details = ''
            if cnum_rej < 15:  # disabled for brevity
                rejs_details = []
                for kix in crec['exs_info'].keys():
                    exs_info2 = crec['exs_info'][kix]
                    rejs_details.append(sorted(list(exs_info2['rejs'])))
                rejs_details = "[" + "; ".join([", ".join(['%3d' % y for y in x]) for x in rejs_details]) + "]"
            
            conv_str = ''
            if use_liberal:
                conv_str = '; c %s (%3d/%3d)' % ('R' if con_rejects else '-', con_num_rej, con_num_tests)
            
            margin_s = ('margin: %.2f %%' % (100.0 * (1 - rnd/max_rounds_cur))) if cur_rejects and max_rounds_cur is not None else '' 
            vdict = '%s (%3d/%3d%s | %s %s max. %s) %s' % ('R' if cur_rejects else '-', cnum_rej, cnum_tests, conv_str, crypt_rounds_cur_s, max_rounds_cur_s, margin_s, rejs_details)
            irnd = rnd
            try:
                irnd = int(rnd)
            except:
                pass
            srnd = ('%4d' if isinstance(irnd, int) else '%4s') % irnd
            
            log_lines.append('%25s | %20s | %s | %s' % (fname, meth, srnd, vdict))
    
        rnd_broken = max_rnd_broken_meth[meth]
        rnd_comp = max_rnd_meth[meth]
        
        # recomp list: start with new rounds only (max computed), upper bound - either max rounds or broken + limit 
        if fname in funcs_no_rounds:
            continue
        recomp_list = list(range(max(rnd_comp + 1, rnd_broken + 1), min(max_rounds_cur + 1, max(rnd_comp + 1, rnd_broken + 1 + exids_recomp_margin)))) 
        exids_recompute[fname][meth] = (cexids, recomp_list)
    
    if max_rnd_broken is None and crypt_rounds_cur is None and max_rounds_cur is None:
        print('Skipping null %s' % fname)
        continue
    
    nfname = fname
    nfname = nfname.replace('ROAD-RUNNER-K80', 'R.RUNNER.K80')
    nfname = nfname.replace('ROAD-RUNNER-K128', 'R.RUNNER.K128')
    nfname = nfname.replace('RECTANGLE-K80', 'RECT.K80')
    nfname = nfname.replace('RECTANGLE-K128', 'RECT.K128')
    
    nfname = nfname.replace('testu01-uxorshift', 'U01.XorShift')
    nfname = nfname.replace('testu01-umrg', 'U01.UMRG')
    nfname = nfname.replace('testu01-ulcg', 'U01.ULCG')
    nfname = nfname.replace('std_subtract_with_carry', 'STD.SubCarry')
    nfname = nfname.replace('std_mersenne_twister', 'STD.MTwister')
    nfname = nfname.replace('std_lcg', 'STD.LCG')
    
    nfname = nfname.replace('mimc_hash-', 'MiMC.')
    nfname = nfname.replace('gmimc-', 'GMiMC.')
    nfname = nfname.replace('lowmc-s', 'LowMC.S')
    nfname = nfname.replace('Vision_', 'Vision.')
    nfname = nfname.replace('Starkad_', 'Starkad.')
    nfname = nfname.replace('Rescue_', 'Rescue.')
    nfname = nfname.replace('Poseidon_S128_BLS12_138', 'Poseidon.BLS12')
    nfname = nfname.replace('Poseidon_', 'Poseidon.')
    
    cline = '%s & \\boxSecMarginXX{%s}{%s}{%s}' % (nfname, max_rnd_broken or 0, max_rounds_cur or 1, crypt_rounds_cur or 0)
    table_lines.append((cline, erec))
    
    #if len(table_lines)> 10: break

    
print("\n".join(log_lines))

cols = 3
per_col = math.ceil(len(table_lines)/3.0)
sorter = lambda x: (x[1].ftype, x[1].stype, natural_sort_key(x[1].fname))
table_lines.sort(key=sorter)
ntable_lines = []

print("\n\nTable lines: ", len(table_lines), ", per col:", per_col)
# for k, g in itertools.groupby(table_lines, lambda x: (x[1].ftype)):

old_type=None
for e in table_lines:
    if (e[1].ftype, e[1].stype) != old_type:
        old_type = (e[1].ftype, e[1].stype)
        if old_type[0] == FuncInfo.HASH:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Hash functions}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 0:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Block ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 2:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Stream ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 4:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{MPC ciphers}}')
        elif old_type[0] == FuncInfo.PRNG:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{PRNGs}}')
        
    ntable_lines.append(e[0])

print(len(ntable_lines))
per_col = math.ceil(len(ntable_lines)/3.0)
arcols = [ntable_lines[i*per_col: (i+1)*per_col] for i in range(cols)]

for x in range(per_col):
    tup = arcols[0][x] if x < len(arcols[0]) else '& ', arcols[1][x] if x < len(arcols[1]) else '& ', arcols[2][x] if x < len(arcols[2]) else '& '
    print("& ".join(tup) + "\\\\")

#for x in range(per_col):
#    l1 =    
#print("\n".join(ntable_lines))

# Specs for adding new rounds to compute
print('-----\n\n\n', json.dumps(exids_recompute))
            

                   Abacus |         ctr:10485760 |    0 | - (  2/856; c - (  0/214) |  -  280 max. ) [;  77; 449]
                   Abacus |         ctr:10485760 |    1 | - (  2/856; c - (  0/214) |  -  280 max. ) [194; ; 191]
                   Abacus |         ctr:10485760 |    2 | - (  4/856; c - (  0/214) |  -  280 max. ) [111, 449;  11;  59]
                   Abacus |         ctr:10485760 |    3 | - (  1/856; c - (  0/214) |  -  280 max. ) [449; ; ]
                   Abacus |        ctr:104857600 |    0 | - (  4/980; c - (  0/245) |  -  280 max. ) [224, 342, 444, 480; ; ]
                   Abacus |        ctr:104857600 |    1 | - (  1/980; c - (  0/245) |  -  280 max. ) [; 471; ]
                   Abacus |        ctr:104857600 |    2 | - (  1/980; c - (  0/245) |  -  280 max. ) [ 39; ; ]
                   Abacus |        ctr:104857600 |    3 | - (  2/980; c - (  0/245) |  -  280 max. ) [448, 510; ; ]
                   Abacus |          hw:10485760 |    0 | - (  2/856; c - (